In [1]:
import torch
from transformers import (
    AutoModelForCausalLM,
    LlamaTokenizerFast,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig,
)
from peft import (
    LoraConfig,
    get_peft_model,
)

2024-04-18 23:47:03.892800: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-18 23:47:04.104870: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-18 23:47:04.871413: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2024-04-18 23:47:04.871573: W tensorflow/compiler/xla/stream_exe

In [2]:
# https://huggingface.co/meta-llama/Llama-2-7b-hf
# base_model = "meta-llama/Llama-2-7b-hf"
base_model = "meta-llama/Llama-2-7b-chat-hf"
# base_model="beomi/open-llama-2-ko-7b"
# base_model = "huggingface-projects/llama-2-7b-chat"
# base_model = "TinyPixel/Llama-2-7B-bf16-sharded"

In [3]:
# # Fine-tuned model
# new_model = "llama-2-7b-hf-fine-tuned-test1"

In [3]:
# QLoRA 모델을 사용하기 위한 설정

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 
)



In [4]:
# 라마2 모델 불러오기
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# 토크나이저 로드
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false" # 토크나이저 병렬처리 방지(오류 방지)
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true' # __cell__ 오류 방지

tokenizer = LlamaTokenizerFast.from_pretrained(
    base_model,
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token # 패딩 토큰을 문장의 끝으로 설정 </s>
tokenizer.padding_side = "right" # 패딩을 문장 뒤에 추가

In [6]:
# 학습 양식
import json
from datasets import load_dataset
file_name = 'preprocess/dataset3/new_ratings_4.json'

In [7]:
import json

# JSON 파일 불러오기
input_file = file_name
with open(input_file, 'r') as f:
    data = json.load(f)

# 불러온 데이터 확인
# print(data)

In [8]:
# print(data[1])

print(data[0]["ratings"]) # 영화 평점들 정보 모음
print(type(data[0]["ratings"])) 

print(data[0]["ratings"][0]) # 평점 1개
print(type(data[0]["ratings"][0])) 
print(data[0]["rank"])

[{'title': 'Toy Story (1995)', 'rating': 4.0, 'imdbId': 'tt114709', 'timestamp': 964982703}, {'title': 'Grumpier Old Men (1995)', 'rating': 4.0, 'imdbId': 'tt113228', 'timestamp': 964981247}, {'title': 'Heat (1995)', 'rating': 4.0, 'imdbId': 'tt113277', 'timestamp': 964982224}, {'title': 'Seven (a.k.a. Se7en) (1995)', 'rating': 5.0, 'imdbId': 'tt114369', 'timestamp': 964983815}, {'title': 'Usual Suspects, The (1995)', 'rating': 5.0, 'imdbId': 'tt114814', 'timestamp': 964982931}, {'title': 'From Dusk Till Dawn (1996)', 'rating': 3.0, 'imdbId': 'tt116367', 'timestamp': 964982400}, {'title': 'Bottle Rocket (1996)', 'rating': 5.0, 'imdbId': 'tt115734', 'timestamp': 964980868}, {'title': 'Braveheart (1995)', 'rating': 4.0, 'imdbId': 'tt112573', 'timestamp': 964982176}, {'title': 'Rob Roy (1995)', 'rating': 5.0, 'imdbId': 'tt114287', 'timestamp': 964984041}, {'title': 'Canadian Bacon (1995)', 'rating': 5.0, 'imdbId': 'tt109370', 'timestamp': 964984100}]
<class 'list'>
{'title': 'Toy Story (1

In [9]:
instruction = '''
I will recommend the list of movies sorted according to user ratings information. And I will show the metadata for the corresponding movie. It must return in JSON format.
'''

In [10]:
# 데이터 매핑 함수 정의
def map_data_to_format(example):

    user_ratings_str = json.dumps(example['ratings'], ensure_ascii=False)
    rank_list_str = json.dumps(example["rank"], ensure_ascii=False)
    
    text = (
        f"###instruction:\n{instruction}\n\n"
        f"user_ratings_info:\n{user_ratings_str}\n\n"
        f"###rank:\n{rank_list_str}\n\n"
    )
    
    # completion은 모델이 생성해야 할 예상 출력을 포함함
    # 여기서는 우선순위에 따라 정렬된 영화 imdb_id의 리스트를 반환합니다.
    completion = f"{{\"rank\": {rank_list_str}}}"
    
    return {'text': text, 'completion': completion}

In [11]:
# 데이터셋 로드
data = load_dataset('json', data_files=file_name, split="train")

In [12]:
print(data)

Dataset({
    features: ['userId', 'ratings', 'rank'],
    num_rows: 10084
})


In [15]:
# print(data['ratings'])


In [16]:
# print(data['order']) # [['tt109370', 'tt114287', 'tt114369', 'tt114814', 'tt115734', 'tt114709', 'tt113277', 'tt112573', 'tt113228', 'tt116367'], ...]

In [17]:
# print(data['userId']) # [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, ...]

In [13]:
# 데이터 매핑 적용
mapped_data = data.map(map_data_to_format)

In [14]:
print(mapped_data)


Dataset({
    features: ['userId', 'ratings', 'rank', 'text', 'completion'],
    num_rows: 10084
})


In [15]:
# 데이터셋 분할
split_data = mapped_data.train_test_split(test_size=0.1)  # 10%를 테스트셋으로 사용

train_set = split_data['train']
eval_set = split_data['test']

train_set = train_set.map(lambda samples: tokenizer(samples["text"], padding=True, truncation=True, return_tensors="pt"), batched=True)
eval_set = eval_set.map(lambda samples: tokenizer(samples["text"], padding=True, truncation=True, return_tensors="pt"), batched=True)

# lora 파라미터 설정
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)


Map:   0%|          | 0/9075 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1009 [00:00<?, ? examples/s]

In [16]:
print(train_set[3]["text"])
print("\n")
print(type(train_set[3]["text"]))
print("\n")
# print(train_set[20]["text"])
print(train_set[120]["text"])

###instruction:

I will recommend the list of movies sorted according to user ratings information. And I will show the metadata for the corresponding movie. It must return in JSON format.


user_ratings_info:
[{"imdbId": "tt119488", "rating": 5.0, "timestamp": 1013394501, "title": "L.A. Confidential (1997)"}, {"imdbId": "tt119468", "rating": 5.0, "timestamp": 1019584596, "title": "Kiss the Girls (1997)"}, {"imdbId": "tt119164", "rating": 5.0, "timestamp": 1013395394, "title": "Full Monty, The (1997)"}, {"imdbId": "tt118971", "rating": 4.0, "timestamp": 1013398024, "title": "The Devil's Advocate (1997)"}, {"imdbId": "tt119177", "rating": 5.0, "timestamp": 1013395308, "title": "Gattaca (1997)"}, {"imdbId": "tt83131", "rating": 5.0, "timestamp": 1019585417, "title": "Stripes (1981)"}, {"imdbId": "tt120201", "rating": 5.0, "timestamp": 1019580795, "title": "Starship Troopers (1997)"}, {"imdbId": "tt119395", "rating": 5.0, "timestamp": 1019581043, "title": "Jackal, The (1997)"}, {"imdbId": 

In [17]:
print(eval_set[3]["text"])
# print(eval_set[20]["text"])
print(eval_set[120]["text"])

###instruction:

I will recommend the list of movies sorted according to user ratings information. And I will show the metadata for the corresponding movie. It must return in JSON format.


user_ratings_info:
[{"imdbId": "tt120744", "rating": 2.5, "timestamp": 1237713993, "title": "Man in the Iron Mask, The (1998)"}, {"imdbId": "tt120632", "rating": 2.0, "timestamp": 1237851732, "title": "City of Angels (1998)"}, {"imdbId": "tt127723", "rating": 2.5, "timestamp": 1237713979, "title": "Can't Hardly Wait (1998)"}, {"imdbId": "tt120591", "rating": 2.0, "timestamp": 1237710564, "title": "Armageddon (1998)"}, {"imdbId": "tt138704", "rating": 3.5, "timestamp": 1237851228, "title": "Pi (1998)"}, {"imdbId": "tt129387", "rating": 3.0, "timestamp": 1237712921, "title": "There's Something About Mary (1998)"}, {"imdbId": "tt75148", "rating": 2.0, "timestamp": 1237713426, "title": "Rocky (1976)"}, {"imdbId": "tt95953", "rating": 3.5, "timestamp": 1237857063, "title": "Rain Man (1988)"}, {"imdbId": 

In [18]:
print(train_set[3]["completion"])
# print(train_set[20]["completion"])
print(train_set[120]["completion"])

{"rank": [{"title": "Stripes (1981)"}, {"title": "Kiss the Girls (1997)"}, {"title": "Scream 2 (1997)"}, {"title": "Jackal, The (1997)"}, {"title": "Starship Troopers (1997)"}, {"title": "Full Monty, The (1997)"}, {"title": "Gattaca (1997)"}, {"title": "L.A. Confidential (1997)"}, {"title": "Tomorrow Never Dies (1997)"}, {"title": "The Devil's Advocate (1997)"}]}
{"rank": [{"title": "Dear Frankie (2004)"}, {"title": "Freaks (1932)"}, {"title": "Two Brothers (Deux frères) (2004)"}, {"title": "Dr. Jekyll and Mr. Hyde (1931)"}, {"title": "Anna Christie (1930)"}, {"title": "Christmas Carol, A (Scrooge) (1951)"}, {"title": "Picture of Dorian Gray, The (1945)"}, {"title": "Show Boat (1951)"}, {"title": "Please Don't Eat the Daisies (1960)"}, {"title": "Memphis Belle (1990)"}]}


In [19]:
model = get_peft_model(model, peft_params)

# prameter
epochs = 3 # 10
batch_size = 2
lr = 2e-4

training_params = TrainingArguments(
    output_dir="models4",
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=16,
    optim="adamw_torch",
    save_strategy="epoch",
    evaluation_strategy="steps",
    logging_strategy="steps",
    eval_steps=20,
    logging_steps=20,
    learning_rate=lr,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    dataloader_num_workers=1,
)


In [20]:
# ! wandb login --relogin
# torch.cuda.empty_cache()
# import gc
# gc.collect()
# import gc
# torch.cuda.empty_cache()
# gc.collect()

In [21]:
import transformers
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

trainer = Trainer(
    model=model,
    args=training_params,
    train_dataset=train_set,
    eval_dataset=eval_set,
    tokenizer=tokenizer,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

trainer.train()

/home/gpuadmin/anaconda3/envs/llm_test/lib/python3.8/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


  0%|          | 0/849 [00:00<?, ?it/s]

{'loss': 0.9255, 'grad_norm': 0.06926549226045609, 'learning_rate': 0.00015384615384615385, 'epoch': 0.07}


  0%|          | 0/127 [00:00<?, ?it/s]

{'eval_loss': 0.778734564781189, 'eval_runtime': 249.3284, 'eval_samples_per_second': 4.047, 'eval_steps_per_second': 0.509, 'epoch': 0.07}
{'loss': 0.6226, 'grad_norm': 0.041824668645858765, 'learning_rate': 0.00019985723465913057, 'epoch': 0.14}


...

  0%|          | 0/127 [00:00<?, ?it/s]

{'eval_loss': 0.5098575353622437, 'eval_runtime': 248.2058, 'eval_samples_per_second': 4.065, 'eval_steps_per_second': 0.512, 'epoch': 2.89}
{'loss': 0.5135, 'grad_norm': 0.21717651188373566, 'learning_rate': 5.900820203481594e-08, 'epoch': 2.96}


  0%|          | 0/127 [00:00<?, ?it/s]

{'eval_loss': 0.5097571611404419, 'eval_runtime': 249.7982, 'eval_samples_per_second': 4.039, 'eval_steps_per_second': 0.508, 'epoch': 2.96}
{'train_runtime': 34593.2217, 'train_samples_per_second': 0.787, 'train_steps_per_second': 0.025, 'train_loss': 0.5345927902329516, 'epoch': 2.99}


TrainOutput(global_step=849, training_loss=0.5345927902329516, metrics={'train_runtime': 34593.2217, 'train_samples_per_second': 0.787, 'train_steps_per_second': 0.025, 'train_loss': 0.5345927902329516, 'epoch': 2.99})

In [22]:
from tensorboard import notebook
log_dir = "./models4" 
notebook.start("--logdir {} --port 4000".format(log_dir))

Launching TensorBoard...

In [23]:
instruction = "I will recommend the list of movies sorted according to user ratings information. And I will show the metadata for the corresponding movie. It must return in JSON format."

user_ratings_str = [{"imdbId": "tt199725", "rating": 3.0, "timestamp": 1044311397, "title": "Love and Basketball (2000)"}, {"imdbId": "tt171359", "rating": 3.0, "timestamp": 1044311426, "title": "Hamlet (2000)"}, {"imdbId": "tt187393", "rating": 3.0, "timestamp": 1044311108, "title": "Patriot, The (2000)"}, {"imdbId": "tt181875", "rating": 4.0, "timestamp": 1044311358, "title": "Almost Famous (2000)"}, {"imdbId": "tt180093", "rating": 5.0, "timestamp": 1044311310, "title": "Requiem for a Dream (2000)"}, {"imdbId": "tt120917", "rating": 4.0, "timestamp": 1044311744, "title": "Emperor's New Groove, The (2000)"}, {"imdbId": "tt181865", "rating": 5.0, "timestamp": 1044311310, "title": "Traffic (2000)"}, {"imdbId": "tt209144", "rating": 5.0, "timestamp": 1044311318, "title": "Memento (2000)"}, {"imdbId": "tt125022", "rating": 2.0, "timestamp": 1044311195, "title": "Heartbreakers (2001)"}, {"imdbId": "tt203009", "rating": 4.0, "timestamp": 1044311949, "title": "Moulin Rouge (2001)"}]

# order_list_str=["tt118971", "tt118883", "tt120102", "tt119488", "tt118884", "tt119345", "tt119174", "tt120177", "tt118842", "tt120399"]

In [25]:
# ###instruction:
# {instruction}

text = f'''
###instruction:
{instruction}


user_ratings_info:
{user_ratings_str}


'''
inputs = tokenizer(text, return_tensors="pt").to("cuda")
outputs = model.generate(
    input_ids=inputs["input_ids"].to("cuda"), 
    attention_mask=inputs["attention_mask"], 
    max_new_tokens=256,
    early_stopping=True,
    pad_token_id=tokenizer.eos_token_id
)
output = tokenizer.decode(outputs[0])
print(output)


<s> 
###instruction:
I will recommend the list of movies sorted according to user ratings information. And I will show the metadata for the corresponding movie. It must return in JSON format.


user_ratings_info:
[{'imdbId': 'tt199725', 'rating': 3.0, 'timestamp': 1044311397, 'title': 'Love and Basketball (2000)'}, {'imdbId': 'tt171359', 'rating': 3.0, 'timestamp': 1044311426, 'title': 'Hamlet (2000)'}, {'imdbId': 'tt187393', 'rating': 3.0, 'timestamp': 1044311108, 'title': 'Patriot, The (2000)'}, {'imdbId': 'tt181875', 'rating': 4.0, 'timestamp': 1044311358, 'title': 'Almost Famous (2000)'}, {'imdbId': 'tt180093', 'rating': 5.0, 'timestamp': 1044311310, 'title': 'Requiem for a Dream (2000)'}, {'imdbId': 'tt120917', 'rating': 4.0, 'timestamp': 1044311744, 'title': "Emperor's New Groove, The (2000)"}, {'imdbId': 'tt181865', 'rating': 5.0, 'timestamp': 1044311310, 'title': 'Traffic (2000)'}, {'imdbId': 'tt209144', 'rating': 5.0, 'timestamp': 1044311318, 'title': 'Memento (2000)'}, {'imdb